In [1]:
%run ../../src/reload.py
%run ../../scripts/3_parse_transactions.py

  0%|                                                                                                                                                           | 0/30 [00:00<?, ?it/s]

Filtering NFT, ArtHouse and money ops...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:20<00:00,  1.45it/s]



Writing addrs log...
written 65856385 bytes 390047 entries
Writing transactions info...
written 14445309 bytes 137562 entries
Parsing NFT ops...
written 22967551 bytes 121879 entries
Parsing ArtHouse ops...
written 11299057 bytes 117811 entries
Parsing money ops...


AssertionError: 

## NFT parser

In [9]:
%run ../../src/reload.py
%run ../../src/contracts/nft_parser.py

ops = nft_ops

In [10]:
    calls_counter = Counter()
    status_counter = Counter()
    errors_counter = Counter()
    senders_counter = Counter()

    nft_state = src.contracts.nft_state.NFTState()
    nft_state = src.contracts.state_utils.StateRecorder(nft_state)

    for op in ops:
        op_hash = op['hash']
        op_row_id = op['row_id']
        assert type(op_row_id) is int

        op_status = op['status']
        status_counter[op_status] += 1

        if op_status == 'applied':
            pass

        elif op_status in ['backtracked', 'skipped']:
            continue

        elif op_status == 'failed':
            for error in op['errors']:
                assert error['kind'] == 'temporary'
                assert error['id'] in [
                    'proto.008-PtEdo2Zk.michelson_v1.runtime_error',
                    'proto.008-PtEdo2Zk.gas_exhausted.operation',
                ]
                errors_counter[error['id']] += 1

            continue

        else:
            raise Exception(f'Unknown op status = {op_status}, hash = {op_hash}')

        assert op['volume'] == 0
        senders_counter[op['sender']] += 1

        op_type = op['type']
        if op_type == 'origination':
            assert op is ops[0]
            continue

        assert op_type == 'transaction'

        params = op['parameters']
        call = params['call']
        calls_counter[call] += 1
        value = params['value']

        storage = op['storage']
        assert set(storage.keys()) == {'value'}
        storage_value = storage['value']

        assert set(storage_value.keys()) == {
            'administrator', 'all_tokens',
            'ledger', 'metadata', 'operators',
            'paused', 'token_metadata',
        }
        assert storage_value['administrator'] == src.config.name2addr['art_house_contract']
        assert int(storage_value['all_tokens']) >= 0
        assert storage_value['ledger'] == '511'
        assert storage_value['metadata'] == '512'
        assert storage_value['operators'] == '513'
        assert storage_value['token_metadata'] == '514'
        assert storage_value['paused'] == 'false'

        if call == 'set_administrator':
            assert op is ops[1]
            continue

        elif call == 'update_operators':
            continue

        assert params['entrypoint'] == params['call']

        if call == 'transfer':
            assert op['entrypoint_id'] == 6
            assert params['branch'] == 'RRL'
            assert params['id'] == 6
            assert set(value.keys()) == {'transfer'}
            assert type(value['transfer']) is list
            assert len(value['transfer']) == 1
            assert set(value['transfer'][0].keys()) == {'from_', 'txs'}
            assert value['transfer'][0]['from_']
            assert len(value['transfer'][0]['txs']) >= 1

            txs = []
            for tx in value['transfer'][0]['txs']:
                assert set(tx.keys()) == {'amount', 'to_', 'token_id'}
                txs.append({
                    'from': value['transfer'][0]['from_'],
                    'to': tx['to_'],
                    'token_id': tx['token_id'],
                    'count': tx['amount'],
                })

            diff_set = nft_state.apply_transfer(
                row_id=op_row_id,
                txs=txs,
            )

            if len(diff_set) == 0:
                assert 'big_map_diff' not in op
                continue

            big_map_diff = op['big_map_diff']
            assert len(big_map_diff) == len(diff_set)
            big_map_diff_set = set()
            for big_map_diff_item in big_map_diff:
                assert big_map_diff_item['action'] == 'update'
                big_map_diff_set.add((
                    big_map_diff_item['key']['0@address'],
                    int(big_map_diff_item['key']['1@nat']),
                    int(big_map_diff_item['value']),
                ))
            assert big_map_diff_set == diff_set

        elif call == 'mint':
            assert op['sender'] == src.config.name2addr['art_house_contract']
            assert op['is_internal']
            assert op['entrypoint_id'] == 2
            assert params['branch'] == 'LRL'
            assert params['id'] == 2
            assert set(value.keys()) == {'address', 'amount', 'token_id', 'token_info'}
            assert int(value['amount'])
            assert int(value['token_id'])
            # assert value['address'] == op['creator']
            assert set(value['token_info'].keys()) == {'0@bytes'}
            assert value['token_info']['0@bytes'].startswith('ipfs://')
            assert storage_value['all_tokens'] == str(int(value['token_id']) + 1)

            big_map_diff = op['big_map_diff']
            assert len(big_map_diff) == 2
            assert big_map_diff[0]['key'] == value['token_id']
            assert big_map_diff[0]['action'] == 'update'
            assert big_map_diff[0]['value'] == {
                'token_id': value['token_id'],
                'token_info': value['token_info'],
            }
            assert big_map_diff[1]['key'] == {
                '0@address': value['address'],
                '1@nat': value['token_id'],
            }
            assert big_map_diff[1]['action'] == 'update'
            assert big_map_diff[1]['value'] == value['amount']
            nft_state.apply_mint(
                row_id=op_row_id,
                token_id=int(value['token_id']),
                count=int(value['amount']),
                creator=op['creator'],
                tokens_receiver=value['address'],
                info_ipfs=value['token_info']['0@bytes'],
            )

        else:
            raise Exception(f'Unknown call: {call}')


## Art house parser

In [8]:
%run ../../src/reload.py
%run ../../src/contracts/art_house_parser.py

## Money parser

In [2]:
%run ../../src/reload.py
%run ../../src/contracts/money_parser.py

trs = money_trs

In [27]:
    money_state = src.contracts.money_state.MoneyState()
    money_state = src.contracts.state_utils.StateRecorder(money_state)

    for tr in trs:

        nft_calls = Counter()
        last_nft_op = None
        for op in tr['ops']:
            if op['type'] == 'transaction' and op['receiver'] == src.config.name2addr['nft_contract']:
                nft_calls[op['parameters']['call']] += 1
                last_nft_op = op
        nft_calls = dict(nft_calls)

        money_delta = src.tr.utils.get_tr_money_delta(tr)
        known_money_delta = {
            src.config.addr2name[addr]: delta
            for addr, delta in money_delta.items()
            if addr in src.config.addr2name
        }

        if nft_calls == {'transfer': 1}:

            tr_ops = tr['ops']

            first_contract = tr['ops'][0]['receiver']
            first_call = tr['ops'][0]['parameters']['call']

            if (first_contract, first_call) in [
                ('KT1MMLb2FVrrE9Do74J3FH1RNNc4QhDuVCNX', 'launchExchange'),
                ('KT1HzEyact4wXoST7hTAGqoKiKkPvAwt4rCR', 'tezToTokenPayment'),
                ('KT1HzEyact4wXoST7hTAGqoKiKkPvAwt4rCR', 'tokenToTezPayment'),
                ('KT1HzEyact4wXoST7hTAGqoKiKkPvAwt4rCR', 'investLiquidity'),
            ]:
                continue

            assert len(tr_ops) in [6]

            if len(tr['ops']) == 7:
                assert tr['ops'][0]['type'] == 'reveal'
                tr_ops = tr['ops'][1:]

            assert len(tr_ops) == 6

            # get buyer money
            assert tr_ops[0]['parameters']['call'] == 'collect'
            assert tr_ops[0]['receiver'] == src.config.name2addr['art_house_contract']
            assert tr_ops[0]['volume'] > 0
            payer = tr_ops[0]['sender']
            price = tr_ops[0]['volume']

            # pay royalties
            assert 'parameters' not in tr_ops[1]
            assert tr_ops[1]['volume'] > 0
            royalties_receiver = tr_ops[1]['receiver']
            royalties = tr_ops[1]['volume']

            # pay comission
            assert 'parameters' not in tr_ops[2]
            assert tr_ops[2]['receiver'] == src.config.name2addr['comission_wallet']
            assert tr_ops[2]['volume'] > 0
            comission = tr_ops[2]['volume']

            # pay to seller
            assert 'parameters' not in tr_ops[3]
            assert tr_ops[3]['volume'] > 0
            seller = tr_ops[3]['receiver']
            seller_income = tr_ops[3]['volume']

            # give hDAO tokens
            assert tr_ops[4]['receiver'] == src.config.name2addr['hdao_contract']
            assert tr_ops[4]['parameters']['call'] == 'hDAO_batch'
            assert tr_ops[4]['volume'] == 0

            # transfer NFT
            assert tr_ops[5]['parameters']['call'] == 'transfer'
            assert tr_ops[5]['receiver'] == src.config.name2addr['nft_contract']
            assert tr_ops[5]['volume'] == 0

            transfer = tr_ops[5]['parameters']['value']['transfer']
            assert len(transfer) == 1
            sender = transfer[0]['from_']

            # when the first transfer between users (with non-zero XTZ volume) will happen,
            # this assert will fail
            assert sender == src.config.name2addr['art_house_contract']

            assert len(transfer[0]['txs']) == 1
            receiver = transfer[0]['txs'][0]['to_']

            token_id = int(transfer[0]['txs'][0]['token_id'])
            token_count = int(transfer[0]['txs'][0]['amount'])

            money_state.apply_swap_sell(
                row_id=last_nft_op['row_id'],
                token_id=token_id,
                token_count=token_count,
                payer=payer,
                price=price,
                seller=seller,
                seller_income=seller_income,
                comission=comission,
                royalties_receiver=royalties_receiver,
                royalties=royalties,
            )
            continue

        assert nft_calls == {}

        if 'parameters' in tr['ops'][0]:
            if tr['ops'][0]['parameters']['call'] in [
                'launchExchange', 'tokenToTezPayment', 'investLiquidity',
                'tezToTokenPayment', 'divestLiquidity',
            ]:
                continue

        known_money_addrs = set(known_money_delta.keys())
        if known_money_addrs in [{'baking_benjamins'}, set()]:
            for op in tr['ops']:
                assert op['receiver'] not in src.config.addr2name
            continue

        assert 'comission_wallet' in known_money_addrs

        if 'baking_benjamins' in known_money_addrs:
            assert known_money_addrs == {'baking_benjamins', 'comission_wallet'}
            assert known_money_delta['baking_benjamins'] < 0
            assert known_money_delta['comission_wallet'] > 0
            assert known_money_delta['comission_wallet'] <= -known_money_delta['baking_benjamins']
            money_state.apply_comission_wallet_baking_percent(
                row_id=tr['ops'][0]['row_id'],
                sender=src.config.name2addr['baking_benjamins'],
                volume=known_money_delta['comission_wallet'],
            )
            continue

        assert known_money_addrs == {'comission_wallet'}
        assert known_money_delta['comission_wallet'] < 0
        other_addrs = set(money_delta.keys()) - {src.config.name2addr['comission_wallet']}
        assert len(other_addrs) == 1
        other_addr = list(other_addrs)[0]
        volume = money_delta[other_addr]
        assert volume > 0
        assert money_delta == {
            src.config.name2addr['comission_wallet']: -volume,
            other_addr: volume
        }
        money_state.apply_comission_wallet_spending(
            row_id=tr['ops'][0]['row_id'],
            receiver=other_addr,
            volume=volume,
        )


AssertionError: 

In [29]:
tr['time']

'2021-04-03T08:56:43Z'

In [4]:
len(tr_ops)

2